## Imports

In [25]:
import torch, torchvision
import numpy as np
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from matplotlib import pyplot as plt 
import torch.nn.functional as F
import torchvision.models as models

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## Load the CIFAR-10 dataset

In [26]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root = "./data", train = True, download = True, transform = transform)
testset = datasets.CIFAR10(root = "./data", train = False, download = True, transform = transform)

batch_size = 64
# You should use as many cores you have on your laptop
num_workers = 8

# Fill in the options for both data loaders. Warning: the training dataloader should shuffle the data
trainloader = torch.utils.data.DataLoader(trainset, num_workers=num_workers, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, num_workers=num_workers, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [27]:
input_dim = (3, 32, 32)
hidden_dim = (16, 16)
output_dim = len(classes)
learning_rate = 0.001
num_epochs = 128

## Neural net architecture (from scratch)

You are free to define any kind of convolutional neural network that you think can solve the classification task.
Remember that convolutional neural networks are usually a combination of the following building blocks:
  * Convolutional layers
  * Pooling layers
  * Linear layers

In [28]:
class ConvNetFromScratch(nn.Module):
    def __init__(self, output_dim):
        super(ConvNetFromScratch, self).__init__()

        # Convolutional Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully Connected Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4096, 128), 
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

## Neural net architecture (transfer learning)
You are free to choose any pre-trained model as an encoder that the PyTorch library offers evaluate and compare its performance to the CNN you have built yourself.

In [29]:
class ConvNetTransferLearning(nn.Module):
    def __init__(self, output_dim):
        super(ConvNetTransferLearning, self).__init__()

        # Choose a pre-trained ResNet model as the encoder
        self.encoder = models.resnet18(pretrained=True)
        
        # Modify the classifier part of ResNet to match your output_dim
        in_features = self.encoder.fc.in_features
        self.encoder.fc = nn.Identity()  # Remove the final fully connected layer

        # Add your own classifier
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, output_dim)
        )

    def forward(self, x):
        # Forward pass through the encoder
        x = self.encoder(x)

        # Forward pass through your classifier
        x = self.classifier(x)

        return x

## Train Both Networks

In [32]:
from tqdm import tqdm

model = ConvNetFromScratch(output_dim).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.89)

vram_cache = []
for inputs, targets in trainloader:
    vram_cache.append((inputs.to(device), targets.to(device)))


def train(model, criterion, optimizer, num_epochs, vram_cache):
    for i in tqdm(range(num_epochs)):
        for inputs, targets in vram_cache:
            model.train(True)
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = F.cross_entropy(outputs, targets)

            print(f"Epoch {i + 1}, Loss: {loss.item():.4f}", end="\r")

            loss.backward()
            optimizer.step()
            
train(model, criterion, optimizer, 16, vram_cache)

  0%|          | 0/16 [00:00<?, ?it/s]

  6%|▋         | 1/16 [00:00<00:13,  1.08it/s]

 12%|█▎        | 2/16 [00:01<00:12,  1.08it/s]

 19%|█▉        | 3/16 [00:02<00:12,  1.07it/s]

 25%|██▌       | 4/16 [00:03<00:11,  1.07it/s]

...

 75%|███████▌  | 12/16 [00:11<00:03,  1.07it/s]

 81%|████████▏ | 13/16 [00:12<00:02,  1.07it/s]

 88%|████████▊ | 14/16 [00:13<00:01,  1.07it/s]

 94%|█████████▍| 15/16 [00:13<00:00,  1.07it/s]

100%|██████████| 16/16 [00:14<00:00,  1.07it/s]

In [33]:
model_2 = ConvNetTransferLearning(output_dim).to(device)
criterion_2 = nn.CrossEntropyLoss().to(device)
optimizer_2 = torch.optim.SGD(model_2.parameters(), lr=learning_rate, momentum=0.89)


train(model_2, criterion_2, optimizer_2, 16, vram_cache)

  0%|          | 0/16 [00:00<?, ?it/s]

  6%|▋         | 1/16 [00:06<01:34,  6.29s/it]

 12%|█▎        | 2/16 [00:12<01:26,  6.18s/it]

 19%|█▉        | 3/16 [00:18<01:19,  6.11s/it]

 25%|██▌       | 4/16 [00:24<01:13,  6.16s/it]

...

 75%|███████▌  | 12/16 [01:13<00:24,  6.10s/it]

 81%|████████▏ | 13/16 [01:19<00:18,  6.11s/it]

 88%|████████▊ | 14/16 [01:25<00:12,  6.11s/it]

 94%|█████████▍| 15/16 [01:31<00:06,  6.12s/it]

100%|██████████| 16/16 [01:38<00:00,  6.13s/it]

## Evaluate Both Networks

In [34]:
def test(model, testloader=testloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation during testing
        for inputs, targets in testloader:
            outputs = model(inputs.to(device))
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets.to(device)).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on the test set: {accuracy:.2f}%")
    
test(model)

Accuracy on the test set: 67.91%


In [35]:
test(model_2)

Accuracy on the test set: 79.82%
